# Deploy Llama 2 7B Chat HF TGI via Amazon SageMaker

Llama 2 is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 70 billion parameters. The fine-tuned LLMs, called Llama-2-Chat, are optimized for dialogue use cases. Llama 2 outperforms other open source language models on many external benchmarks, including reasoning, coding, proficiency, and knowledge tests.


Amazon Web Services (AWS) and Hugging Face have released a new Hugging Face Deep Learning Container (DLC) for inference with Large Language Models (LLMs). This new DLC is powered by Text Generation Inference (TGI), an open source solution for deploying and serving LLMs. TGI enables high-performance text generation using Tensor Parallelism and dynamic batching for popular LLMs like StarCoder, BLOOM, GPT-NeoX, StableLM, Llama, and T5.

In this notebook, we will use Amazon SageMaker Hosting instances Real-Time Inference to deploy `meta-llama/Llama-2-7b-chat-hf` model powered by Text Generation Inference (TGI) on Hugging Face LLM DLC.

## Prerequisite

### Hugging Face Account

You need to have Hugging Face account. Sign Up here https://huggingface.co/join with your email if you do not already have account.

- For seamless access of the models avaialble on Hugging Face especially gated models such as Llama, for fine-tuning and inferencing purposes, you need to have Hugging Face Account to obtain read Access Token.
- After signup, login to visit https://huggingface.co/settings/tokens to create read Access token.

### Request access to the next version of Llama

Use the same email id to obtain permission from meta by visiting this link: https://ai.meta.com/resources/models-and-libraries/llama-downloads/

- The Llama models available via Hugging Face are gated models. The use of Llama model is governed by the Meta license. In order to download the model weights and tokenizer, please visit https://ai.meta.com/resources/models-and-libraries/llama-downloads/ and accept their License before requesting access.
- Within 2 days you might be granted access to use Llama models via a confirmation email with subject: **[Access granted] Your request to access model meta-llama/Llama-2-13b-chat-hf has been accepted.** Though the model id is Llama-2-13b-chat-hf, you should be able to access other variants too.




## Setup Development Environment

Upgrade `pip` and install the latest version of `sagemaker` and `boto3` packages.

In [ ]:
!pip install -Uq pip
!pip install -Uq boto3 sagemaker

Obtain the sagemaker execution role. We need it to create `HuggingFaceModel`.

In [ ]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

## Get the latest Hugging Face LLM Container Image URI

Obtain the latest Hugging Face LLM DLC powered by Text Generation Inference (TGI) available on SageMaker. We will use this image to deploy `meta-llama/Llama-2-7b-chat-hf` model on SageMaker, 

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.0.3"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

## Deploy Llama 2 7b Chat HuggingFace Model

Here is the default recommendation of sagemaker instances when deploying Llama 2 models for inferencing purposes.

| Model Name | Model ID | Minimum Recommended Instance for inference |
|---|---|---|
| Llama-2-7b | meta-llama/Llama-2-7b | ml.g5.2xlarge |
| Llama-2-7b-hf | meta-llama/Llama-2-7b-hf | ml.g5.2xlarge |
| Llama-2-7b-chat | meta-llama/Llama-2-7b-chat-hf | ml.g5.2xlarge |
| Llama-2-13b | meta-llama/Llama-2-13b | ml.g5.12xlarge |
| Llama-2-13b-hf | meta-llama/Llama-2-13b-hf | ml.g5.12xlarge |
| Llama-2-13b-chat | meta-llama/Llama-2-13b-chat-hf | ml.g5.12xlarge |
| Llama-2-70b | meta-llama/Llama-2-70b | ml.g5.48xlarge |
| Llama-2-70b-hf | meta-llama/Llama-2-70b-hf | ml.g5.48xlarge |
| Llama-2-70b-chat | meta-llama/Llama-2-70b-chat-hf | ml.g5.48xlarge |

Reference: [Llama 2 foundation models from Meta are now available in Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/)

We will proceed with deploying `meta-llama/Llama-2-7b-chat-hf` model on `ml.g5.2xlarge`

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.g5.2xlarge"
number_of_gpu = 1
health_check_timeout = 300

# Define Model and Endpoint configuration parameters
config = {
  'HF_MODEL_ID': "meta-llama/Llama-2-7b-chat-hf", # model_id from hf.co/models
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(2048),  # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(4096),  # Max length of the generation (including input text)
  'MAX_BATCH_TOTAL_TOKENS': json.dumps(8192),  # Limits the number of tokens that can be processed in parallel during the generation
  'HUGGING_FACE_HUB_TOKEN': "<YOUR_HUGGING_FACE_READ_ACCESS_TOKEN>" # Read Access token of your HuggingFace profile https://huggingface.co/settings/tokens
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

In [ ]:
from sagemaker.utils import name_from_base

endpoint_name = name_from_base(f"{config['HF_MODEL_ID'].split('/')[1]}-streaming")

Deploy the model to the SageMaker Endpoint

In [ ]:
%%time
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

llm = llm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
)

We will store the value of the variable `endpoint_name` to use it in inference notebook.

In [ ]:
%store \
endpoint_name

## References:
1. [Announcing the launch of new Hugging Face LLM Inference containers on Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/announcing-the-launch-of-new-hugging-face-llm-inference-containers-on-amazon-sagemaker/)
2. [Sagemaker Real-time Inference now supports response streaming](https://aws.amazon.com/about-aws/whats-new/2023/09/sagemaker-real-time-inference-response-streaming/)
3. [Deep learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html)
4. [Hugging Face Gated Models](https://huggingface.co/docs/hub/models-gated)
5. [Hugging Face TGI Open API specification](https://huggingface.github.io/text-generation-inference/)